In [1]:
import gym
import collections 
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
learning_rate=0.0005
gamma=0.98
buffer_limit=5000
batch_size=32

In [3]:
class ReplayBuffer():
    def __init__(self):
        self.buffer=collections.deque(maxlen=buffer_limit)
    
    def put(self,transition):
        self.buffer.append(transition)
    
    def sample(self,n):
        mini_batch=random.sample(self.buffer,n)
        s_lst, a_lst, r_lst,s_prime_lst,done_mask_lst=[],[],[],[],[]
        for transition in mini_batch:
            s,a,r,s_prime,done_mask=transition
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            done_mask_lst.append([done_mask])
        return torch.tensor(s_lst,dtype=torch.float),torch.tensor(a_lst),\
    torch.tensor(r_lst),torch.tensor(s_prime_lst,dtype=torch.float),\
    torch.tensor(done_mask_lst)
    
    def size(self):
        return len(self.buffer)

In [4]:
class Qnet(nn.Module):
    def __init__(self):
        super(Qnet,self).__init__()
        self.fc1=nn.Linear(4,128)
        self.fc2=nn.Linear(128,128)
        self.fc3=nn.Linear(128,2)
    def forward(self,x):
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.fc3(x)
        return x
    def sample_action(self,obs,epsilon):
        out=self.forward(obs)
        coin=random.random()
        if coin<epsilon:
            return random.randint(0,1)
        else:
            return out.argmax().item()

In [5]:
def train(q,q_target,memory,optimizer):
    for i in range(10):
        s,a,r,s_prime,done_mask=memory.sample(batch_size)
        q_out=q(s)
        q_a=q_out.gather(1,a)
        max_q_prime=q_target(s_prime).max(1)[0].unsqueeze(1)
        target=r+gamma*max_q_prime*done_mask
        loss=F.smooth_l1_loss(q_a,target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [7]:
env=gym.make('CartPole-v1')
q=Qnet()
q_target=Qnet()
q_target.load_state_dict(q.state_dict())
memory=ReplayBuffer()

print_interval=20
score=0.0
optimizer=optim.Adam(q.parameters(),lr=learning_rate)

for n_epi in range(1000):
    epsilon=max(0.01,0.08-0.01*(n_epi/20))
    s=env.reset()
    done=False
    while not done:
        a=q.sample_action(torch.from_numpy(s).float(),epsilon)
        s_prime,r,done,info = env.step(a)
        done_mask=0.0 if done else 1.0
        memory.put((s,a,r/100.0,s_prime,done_mask))
        s=s_prime
        score+=r
        if done:
            break
    if memory.size()>200:
        train(q,q_target,memory,optimizer)
    if n_epi%print_interval==0 and n_epi!=0:
        q_target.load_state_dict(q.state_dict())
        print("n_episode :{}, score : {:.1f}, n_buffer : {}, eps : {:.1f}".format(
                                                            n_epi, score/print_interval, memory.size(), epsilon))
        score =0.0
env.close()

c:\users\박정은\gym\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


n_episode :20, score : 10.4, n_buffer : 209, eps : 0.1
n_episode :40, score : 10.1, n_buffer : 410, eps : 0.1
n_episode :60, score : 9.8, n_buffer : 606, eps : 0.1
n_episode :80, score : 10.7, n_buffer : 819, eps : 0.0
n_episode :100, score : 13.3, n_buffer : 1086, eps : 0.0
n_episode :120, score : 18.8, n_buffer : 1462, eps : 0.0
n_episode :140, score : 28.9, n_buffer : 2040, eps : 0.0
n_episode :160, score : 20.2, n_buffer : 2444, eps : 0.0
n_episode :180, score : 13.6, n_buffer : 2715, eps : 0.0
n_episode :200, score : 23.4, n_buffer : 3184, eps : 0.0
n_episode :220, score : 15.4, n_buffer : 3492, eps : 0.0
n_episode :240, score : 156.8, n_buffer : 5000, eps : 0.0
n_episode :260, score : 302.0, n_buffer : 5000, eps : 0.0
n_episode :280, score : 330.2, n_buffer : 5000, eps : 0.0
n_episode :300, score : 281.9, n_buffer : 5000, eps : 0.0
n_episode :320, score : 280.9, n_buffer : 5000, eps : 0.0
n_episode :340, score : 325.1, n_buffer : 5000, eps : 0.0
n_episode :360, score : 329.9, n_b